# Tune GPT2 to generate positive reviews
> Optimise GPT2 to produce positive IMDB movie reviews using a BERT sentiment classifier as a reward function.

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2_bert_training.png' width='600'>
<p style="text-align: center;"> <b>Figure:</b> Experiment setup to tune GPT2. The yellow arrows are outside the scope of this notebook, but the trained models are available through Hugging Face. </p>
</div>


In this notebook we fine-tune GPT2 (small) to generate positive movie reviews based on the IMDB dataset. The model gets the start of a real review and is tasked to produce positive continuations. To reward positive continuations we use a BERT classifier to analyse the sentiment of the produced sentences and use the classifier's outputs as rewards signals for PPO training.

## Setup experiment

### Import dependencies

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
%pip install transformers trl wandb

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

### Configuration

In [ ]:
config = PPOConfig(
    #  model_name="lvwerra/gpt2-imdb", --> ERROR: PPOConfig.__init__() got an unexpected keyword argument 'model_name'. Did you mean 'label_names'? 
    # lvwerra/gpt2-imdb is model from SFT stage (instruction fine tuned or fined tuned on the imdb dataset)
    sft_model_path = "lvwerra/gpt2-imdb",
    #reward_model_path = "lvwerra/distilbert-imdb",
    learning_rate=1.41e-5,
    report_to="wandb",

    # Try disabling this when running on runpod.io
    # bf16=False,  # Explicitly disable bf16
    # fp16=True,   # Use fp16 instead (or remove both for no mixed precision)
)

sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 16}

In [6]:
import wandb

wandb.init()

wandb: Currently logged in as: robdallagogar (robdallagogar-ferraz-h) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


You can see that we load a GPT2 model called `gpt2_imdb`. This model was additionally fine-tuned on the IMDB dataset for 1 epoch with the huggingface [script](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py) (no special settings). The other parameters are mostly taken from the original paper ["Fine-Tuning Language Models from Human Preferences"](
https://arxiv.org/pdf/1909.08593.pdf). This model as well as the BERT model is available in the Huggingface model zoo [here](https://huggingface.co/models). The following code should automatically download the models.

## Load data and models

### Load IMDB dataset
The IMDB dataset contains 50k movie review annotated with "positive"/"negative" feedback indicating the sentiment.  We load the IMDB dataset into a DataFrame and filter for comments that are at least 200 characters. Then we tokenize each text and cut it to random size with the `LengthSampler`.

In [ ]:
def build_dataset(config, dataset_name="imdb", input_min_text_length=2, input_max_text_length=8):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    # Load the tokenizer from the SFT model path (the pre-trained model we'll fine-tune)
    # This ensures we use the same tokenizer that was used to train the base model
    tokenizer = AutoTokenizer.from_pretrained(config.sft_model_path)
    
    # Set the padding token to be the same as the end-of-sequence token
    # This is important for batch processing where sequences have different lengths
    tokenizer.pad_token = tokenizer.eos_token
    
    # Load the IMDB dataset from Hugging Face datasets library
    # We only load the "train" split which contains the training data
    ds = load_dataset(dataset_name, split="train")
    
    # Rename the "text" column to "review" for clarity
    # The original dataset has a column called "text" containing movie reviews
    ds = ds.rename_columns({"text": "review"})
    
    # Filter out reviews that are too short (less than 200 characters)
    # This ensures we have substantial text to work with for training
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)

    # Create a LengthSampler that will randomly choose input lengths between min and max
    # This adds variety to training by using different input sequence lengths
    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        # Convert the text review into token IDs using the tokenizer
        # The [:] slice ensures we don't exceed the randomly chosen input size
        sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
        
        # Convert the token IDs back to text to create the "query" field
        # This query will be used as the starting prompt for the model to continue
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    # Apply the tokenize function to each sample in the dataset
    # This processes all reviews and creates the input_ids and query fields
    ds = ds.map(tokenize, batched=False)
    
    # Convert the dataset to PyTorch format for efficient training
    # This makes the data compatible with PyTorch tensors and DataLoaders
    ds.set_format(type="torch")
    
    # Return the processed dataset ready for training
    return ds

In [ ]:
# Build the dataset using our custom function
dataset = build_dataset(config)


def collator(data):
    """
    Custom data collator for batching samples together.
    
    This function takes a list of individual samples and groups them into batches
    by collecting all values for each key across all samples in the batch.
    
    Args:
        data (list): A list of dictionaries, where each dictionary represents one sample
                    Each sample has keys like 'input_ids', 'query', etc.
    
    Returns:
        dict: A dictionary where each key contains a list of values from all samples
              Example: {'input_ids': [tensor1, tensor2, ...], 'query': [str1, str2, ...]}
    """
    # Get all the keys from the first sample (all samples should have the same keys)
    # data[0] is the first sample in the batch
    # data[0].keys() gives us all the field names like 'input_ids', 'query', etc.
    return dict((key, [d[key] for d in data]) for key in data[0])

tokenizer_config.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/24895 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1168 > 1024). Running this sequence through the model will result in indexing errors


In [39]:
# # Let's take a look at the dataset
# for sample in dataset[:4]:

#     Why it's only printing keys:
#     When you do for sample in dataset[:4]:, 
#     you're iterating over the column names (keys) 
#     of the dataset, not the individual samples. 
#     So sample contains strings like 'input_ids', 'query', etc.

#     print("Sample:")
#     print(sample)
import pprint

for i in range(3):
    sample = dataset[i]
    print(f"Sample {i}:")
    pprint.pprint(sample, indent=2)
    print()  # Add a blank line between samples for readability

Sample 0:
{ 'input_ids': tensor([   40, 26399,   314]),
  'label': tensor(0),
  'query': 'I rented I',
  'review': 'I rented I AM CURIOUS-YELLOW from my video store because of all '
            'the controversy that surrounded it when it was first released in '
            '1967. I also heard that at first it was seized by U.S. customs if '
            'it ever tried to enter this country, therefore being a fan of '
            'films considered "controversial" I really had to see this for '
            'myself.<br /><br />The plot is centered around a young Swedish '
            'drama student named Lena who wants to learn everything she can '
            'about life. In particular she wants to focus her attentions to '
            'making some sort of documentary on what the average Swede thought '
            'about certain political issues such as the Vietnam War and race '
            'issues in the United States. In between asking politicians and '
            'ordinary denizens 

### Load pre-trained GPT2 language models

We load the GPT2 model with a value head and the tokenizer. We load the model twice; the first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This serves as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original language model.

In [41]:
# Load the model twice

# Model to act as the policy
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.sft_model_path)
# Model to act as a reference 
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.sft_model_path)
tokenizer = AutoTokenizer.from_pretrained(config.sft_model_path)

tokenizer.pad_token = tokenizer.eos_token

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

### Initialize PPOTrainer
The `PPOTrainer` takes care of device placement and optimization later on:

In [43]:
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, train_dataset=dataset, data_collator=collator)

TypeError: PPOTrainer.__init__() missing 2 required positional arguments: 'reward_model' and 'value_model'

In [51]:
# First, let's load the BERT model properly for use as reward_model
from transformers import AutoModelForSequenceClassification, AutoTokenizer as BertTokenizer

# Load the BERT model and tokenizer separately (not as pipeline)
bert_model_name = "lvwerra/distilbert-imdb"
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name)

# Determine device BEFORE creating ppo_trainer
device = 0 if torch.cuda.is_available() else "cpu"
bert_model = bert_model.to(device)

# Print BERT model info
print("BERT Model Info:")
print(f"Model: {bert_model}")
print(f"Model config: {bert_model.config}")
print(f"Device: {next(bert_model.parameters()).device}")
print(f"Number of parameters: {sum(p.numel() for p in bert_model.parameters())}")
print()

# Also create the sentiment pipeline for testing (optional)
sentiment_pipe = pipeline("sentiment-analysis", model=bert_model, tokenizer=bert_tokenizer, device=device)

# Test the BERT model with examples
print("Testing BERT model with examples:")
test_texts = [
    "this movie was really bad!!",
    "I love this film, it's amazing!"
]

for text in test_texts:
    result = sentiment_pipe(text, **sent_kwargs)
    print(f"Text: '{text}'")
    print(f"Sentiment scores: {result}")
    print()

Device set to use cpu


BERT Model Info:
Model: DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropou

### Load BERT classifier
We load a BERT classifier fine-tuned on the IMDB dataset.

In [44]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug
sentiment_pipe = pipeline("sentiment-analysis", model="lvwerra/distilbert-imdb", device=device)

NameError: name 'ppo_trainer' is not defined

The model outputs are the logits for the negative and positive class. We will use the logits for positive class as a reward signal for the language model.

In [11]:
text = "this movie was really bad!!"
sentiment_pipe(text, **sent_kwargs)

/raid/sourab/miniconda3/envs/hf/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'NEGATIVE', 'score': 2.3350484371185303},
  {'label': 'POSITIVE', 'score': -2.726576328277588}]]

In [12]:
text = "this movie was really good!!"
sentiment_pipe(text, **sent_kwargs)

[[{'label': 'NEGATIVE', 'score': -2.294790267944336},
  {'label': 'POSITIVE', 'score': 2.557040214538574}]]

# *BUILDING PPOTRAINER WITH THE NEWER VERSION OF TRL!!*

**1. Load BERT Classifier**

**2. Initialize the PPOTrainer**

In [52]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer as BertTokenizer

# Load the BERT model and tokenizer separately (not as pipeline)
bert_model_name = "lvwerra/distilbert-imdb"
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name)

# Determine device BEFORE creating ppo_trainer
device = 0 if torch.cuda.is_available() else "cpu"
bert_model = bert_model.to(device)

# Print BERT model info
print("BERT Model Info:")
print(f"Model: {bert_model}")
print(f"Model config: {bert_model.config}")
print(f"Device: {next(bert_model.parameters()).device}")
print(f"Number of parameters: {sum(p.numel() for p in bert_model.parameters())}")
print()

# Also create the sentiment pipeline for testing (optional)
sentiment_pipe = pipeline("sentiment-analysis", model=bert_model, tokenizer=bert_tokenizer, device=device)

# Test the BERT model with examples
print("Testing BERT model with examples:")
test_texts = [
    "this movie was really bad!!",
    "I love this film, it's amazing!"
]

for text in test_texts:
    result = sentiment_pipe(text, **sent_kwargs)
    print(f"Text: '{text}'")
    print(f"Sentiment scores: {result}")
    print()

Device set to use cpu


BERT Model Info:
Model: DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropou

In [57]:
model.pretrained_model.generation_config 

GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256
}

In [58]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer as BertTokenizer

# Load the BERT model and tokenizer separately (not as pipeline)
bert_model_name = "lvwerra/distilbert-imdb"
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name)

# Determine device BEFORE creating ppo_trainer
device = 0 if torch.cuda.is_available() else "cpu"
bert_model = bert_model.to(device)

# Print BERT model info
print("BERT Model Info:")
print(f"Model: {bert_model}")
print(f"Model config: {bert_model.config}")
print(f"Device: {next(bert_model.parameters()).device}")
print(f"Number of parameters: {sum(p.numel() for p in bert_model.parameters())}")
print()

# Also create the sentiment pipeline for testing (optional)
sentiment_pipe = pipeline("sentiment-analysis", model=bert_model, tokenizer=bert_tokenizer, device=device)

# Test the BERT model with examples
print("Testing BERT model with examples:")
test_texts = [
    "this movie was really bad!!",
    "I love this film, it's amazing!"
]

for text in test_texts:
    result = sentiment_pipe(text, **sent_kwargs)
    print(f"Text: '{text}'")
    print(f"Sentiment scores: {result}")
    print()

# IMPORTANT FIX: Ensure the models have generation_config properly set
# The AutoModelForCausalLMWithValueHead accesses generation_config through pretrained_model
if not hasattr(model, 'generation_config'):
    if hasattr(model, 'pretrained_model') and hasattr(model.pretrained_model, 'generation_config'):
        model.generation_config = model.pretrained_model.generation_config
    else:
        # Create a default generation config if it doesn't exist
        from transformers import GenerationConfig
        model.generation_config = GenerationConfig()

if not hasattr(ref_model, 'generation_config'):
    if hasattr(ref_model, 'pretrained_model') and hasattr(ref_model.pretrained_model, 'generation_config'):
        ref_model.generation_config = ref_model.pretrained_model.generation_config
    else:
        from transformers import GenerationConfig
        ref_model.generation_config = GenerationConfig()

# Now initialize PPOTrainer with correct parameters
ppo_trainer = PPOTrainer(
    config, 
    tokenizer, 
    model, 
    ref_model, 
    reward_model=bert_model,  # Use the actual BERT model
    train_dataset=dataset, 
    value_model=model,  # Same as policy model since it has value head
    data_collator=collator
)

# Print PPOTrainer info
print("PPOTrainer Info:")
print(f"PPOTrainer device: {ppo_trainer.accelerator.device}")
print(f"Policy model device: {next(model.parameters()).device}")
print(f"Reference model device: {next(ref_model.parameters()).device}")
print(f"Reward model device: {next(bert_model.parameters()).device}")
print(f"Value model device: {next(model.parameters()).device}")  # Same as policy model

Device set to use cpu


BERT Model Info:
Model: DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropou

/Users/robertagarcia/Desktop/GenAI/Training LLMs from Scratch/training-llms-from-scratch/.venv/lib/python3.13/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


ValueError: fp16 mixed precision requires a GPU (not 'mps').

### Generation settings
For the response generation we just use sampling and make sure top-k and nucleus sampling are turned off as well as a minimal length.

In [13]:
gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0, "do_sample": True, "pad_token_id": tokenizer.eos_token_id}

## Optimize model

### Training loop

The training loop consists of the following main steps:
1. Get the query responses from the policy network (GPT-2)
2. Get sentiments for query/responses from BERT
3. Optimize policy with PPO using the (query, response, reward) triplet

**Training time**

This step takes **~2h** on a V100 GPU with the above specified settings.

In [14]:
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)


generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

0it [00:00, ?it/s]/raid/sourab/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (32.40) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/raid/sourab/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (11.56) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/raid/sourab/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (21.80) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/raid/sourab/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (27.79) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/raid/sourab/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (11.69) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/raid/sourab/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (24.06) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/raid/sourab/trl/trl/trainer/ppo_trainer.py:12

### Training progress
If you are tracking the training progress with Weights&Biases you should see a plot similar to the one below. Check out the interactive sample report on wandb.ai: [link](https://app.wandb.ai/huggingface/trl-showcase/runs/1jtvxb1m/).

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2_tuning_progress.png' width='800'>
<p style="text-align: center;"> <b>Figure:</b> Reward mean and distribution evolution during training. </p>
</div>

One can observe how the model starts to generate more positive outputs after a few optimisation steps.

> Note: Investigating the KL-divergence will probably show that at this point the model has not converged to the target KL-divergence, yet. To get there would require longer training or starting with a higher initial coefficient.

## Model inspection
Let's inspect some examples from the IMDB dataset. We can use `model_ref` to compare the tuned model `model` against the model before optimisation.

In [15]:
#### get a batch from the dataset
bs = 16
game_data = dict()
dataset.set_format("pandas")
df_batch = dataset[:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

/raid/sourab/miniconda3/envs/hf/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/raid/sourab/miniconda3/envs/hf/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,query,response (before),response (after),rewards (before),rewards (after)
0,When originally screened in America in 1972,", this would become a",", this is a &",1.155222,1.224865
1,Born Again,is just awful; her only chased after a nun st...,"is a fun ride, confident adventure with Russi...",-2.930447,2.764369
2,"Generally speaking,",the British would rather that Britain shone a...,I found it interesting and I really enjoyed t...,-0.048785,2.599930
3,I have seen the recent,"improvements of 'Hunter and Madam,",movies of the Long City within as much as,1.404229,1.642995
4,"Description: Corny, utterly stupid",", absurd, Mel Brooks character -- is",", fun and especially good as a mummy",-2.245211,1.383026
5,Director Edward Sed,ol-Smith who was dismayed by this mercy,law is one of my favorite (dr. one,-1.463929,2.364105
6,Robert Siodmak,plays a group of high school student Valerie ...,is a great actor. The story is fantastic prov...,0.475510,2.585483
7,DO NOT,BE JUDGEED!!NOTE,SILENTLY READ!! It's,0.281520,0.485400
8,"Where to start, this",well-done collection could have easily been o...,one of the best movies I've ever heard of the...,2.047390,2.238423
9,STAR RATING:,8/10 deliverers do not set off alarm bells in...,"I love this movie, and the acting. It is trul...",0.642155,2.750093


Looking at the reward mean/median of the generated sequences we observe a significant difference.

In [16]:
print("mean:")
display(df_results[["rewards (before)", "rewards (after)"]].mean())
print()
print("median:")
display(df_results[["rewards (before)", "rewards (after)"]].median())

mean:


rewards (before)    0.065812
rewards (after)     1.953304
dtype: float64


median:


rewards (before)    0.378515
rewards (after)     2.393761
dtype: float64

## Save model
Finally, we save the model and push it to the Hugging Face for later usage.

In [17]:
model.save_pretrained("gpt2-imdb-pos-v2", push_to_hub=True)
tokenizer.save_pretrained("gpt2-imdb-pos-v2", push_to_hub=True)

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

('gpt2-imdb-pos-v2/tokenizer_config.json',
 'gpt2-imdb-pos-v2/special_tokens_map.json',
 'gpt2-imdb-pos-v2/vocab.json',
 'gpt2-imdb-pos-v2/merges.txt',
 'gpt2-imdb-pos-v2/added_tokens.json',
 'gpt2-imdb-pos-v2/tokenizer.json')